In [ ]:
import torch as th
import json
import numpy as np

dat = th.load('./input_latents.pt')
gt = "/data/mint/Motion_Dataset/Mixamo/output_mixamo/Run To Rolling/cam_0/skeleton_cam_0.json"
with open(gt, 'r') as f:
    gt = json.load(f)['joints_3d']
print(dat.shape)
print(np.array(gt).shape)


gt = np.array(gt)

# decoder_layer = nn.TransformerDecoderLayer(d_model=512, nhead=8)
# transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
# memory = torch.rand(10, 32, 512)
# tgt = torch.rand(20, 32, 512)
# out = transformer_decoder(tgt, memory)

torch.Size([1, 16, 19, 64, 64])
(74, 65, 3)
